In [1]:
%load_ext autoreload
%autoreload 2
%matplotlib inline

In [2]:
# load local version of gallifrey, before pip installed version, for debugging
import pathlib
import sys
import os

sys.path.append(str(pathlib.Path(os.getcwd()).parent.joinpath("src")))

In [3]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns
import yt

from gallifrey.analysis import count_planets
from gallifrey.model import Model
from gallifrey.utilities.dataframe import rename_labels
from gallifrey.utilities.math import calculate_pca
from gallifrey.visualization.visualization_utils import (
    FigureProcessor,
    adjust_legend,
    get_palette,
    ridgeplot,
    set_plot_defaults,
)

## DATA SETUP

In [4]:
snapshot = 127
resolution = 8192
sim_id = "37_11"
num_embryos = 50
host_star_masses = 1
planet_categories = [
    "Earth",
    "Super-Earth",
    "Neptunian",
    "Sub-Giant",
    "Giant",
]

In [5]:
model = Model(
    snapshot=snapshot,
    resolution=resolution,
    sim_id=sim_id,
)
ds = model.ds
mw = model.halo

yt : [INFO     ] 2023-11-06 17:21:14,832 Calculating time from 1.000e+00 to be 4.318e+17 seconds
yt : [INFO     ] 2023-11-06 17:21:15,024 Parameters: current_time              = 4.318436660023809e+17 s
yt : [INFO     ] 2023-11-06 17:21:15,025 Parameters: domain_dimensions         = [1 1 1]
yt : [INFO     ] 2023-11-06 17:21:15,027 Parameters: domain_left_edge          = [0. 0. 0.]
yt : [INFO     ] 2023-11-06 17:21:15,029 Parameters: domain_right_edge         = [100. 100. 100.]
yt : [INFO     ] 2023-11-06 17:21:15,030 Parameters: cosmological_simulation   = 1
yt : [INFO     ] 2023-11-06 17:21:15,031 Parameters: current_redshift          = 4.440892098500626e-16
yt : [INFO     ] 2023-11-06 17:21:15,032 Parameters: omega_lambda              = 0.682
yt : [INFO     ] 2023-11-06 17:21:15,033 Parameters: omega_matter              = 0.318
yt : [INFO     ] 2023-11-06 17:21:15,034 Parameters: omega_radiation           = 0.0
yt : [INFO     ] 2023-11-06 17:21:15,034 Parameters: hubble_constant      

## PLOT SETUP

In [6]:
set_plot_defaults()

figure_subdirectory = f"MilkyWayAnalogue/{model.resolution}/{model.sim_id}/{model.snapshot}/Planet_numbers"
save = True

## CREATE DATA SOURCE

In [7]:
radius = 0.2 * mw.virial_radius()

## ABSOLUTE NUMBER PER PLANET TYPE

In [25]:
def aggregate_planet_counts_for_embryos(host_star_mass, embryo_list):
    counts_by_embryos = {
        num_embryos: count_planets(
            model=model,
            data_creator=lambda: mw.sphere(radius=radius),
            planet_categories=planet_categories,
            model_config={
                "ngpps_star_masses": host_star_mass,
                "ngpps_num_embryos": num_embryos,
            },
            components="stars",
        )
        for num_embryos in embryo_list
    }
    counts_dataframe = pd.concat(counts_by_embryos).reset_index(level=1, drop=True)
    counts_dataframe["Total"] = counts_dataframe.sum(axis=1)
    return counts_dataframe.rename_axis(r"$N_\mathrm{Embryos}$")

In [26]:
embryo_list = (10, 20, 50, 100)

planet_counts = aggregate_planet_counts_for_embryos(
    host_star_mass=host_star_masses, embryo_list=embryo_list
)


      CONFIG: Setting ngpps_num_embryos to 10.


      TIMER: Block 'Adding Planets...' took 2.62 seconds.


      CONFIG: Setting ngpps_num_embryos to 20.


      TIMER: Block 'Adding Planets...' took 3.13 seconds.


      CONFIG: Setting ngpps_num_embryos to 50.


      TIMER: Block 'Adding Planets...' took 4.37 seconds.


      CONFIG: Setting ngpps_num_embryos to 100.


      TIMER: Block 'Adding Planets...' took 5.7 seconds.



In [27]:
print(
    "ALSO DO FOR OTHER MASSES, DISCUSS THE OBSERVATIONAL ARGUMENTS (EARTHS MORE COMMON THAN SUPER-EARTHS -> NOT LIKE IN OBSERVATIONS, PLANETS AROUND DWARFS RARER -> NOT IN OBSERVATIONS)"
)
print("COMPARE AND RESCALE TO OBSERVATIONAL VALUES (PLATO PAPER MATUSZEWSKI2023")
planet_counts

ALSO DO FOR OTHER MASSES, DISCUSS THE OBSERVATIONAL ARGUMENTS (EARTHS MORE COMMON THAN SUPER-EARTHS -> NOT LIKE IN OBSERVATIONS, PLANETS AROUND DWARFS RARER -> NOT IN OBSERVATIONS)
COMPARE AND RESCALE TO OBSERVATIONAL VALUES (PLATO PAPER MATUSZEWSKI2023


,Earth,Super-Earth,Neptunian,Sub-Giant,Giant,Total
$N_\mathrm{Embryos}$,,,,,,
10,3.046638e+09,3.732298e+09,1.255152e+09,3.509185e+08,8.238287e+08,9.208835e+09
20,5.178461e+09,5.144766e+09,1.363941e+09,3.759609e+08,8.226259e+08,1.288575e+10
50,9.363212e+09,9.088388e+09,1.315451e+09,2.934588e+08,7.104246e+08,2.077093e+10
100,1.100211e+10,1.097044e+10,1.420060e+09,3.227412e+08,7.377153e+08,2.445307e+10
